# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
% matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import scipy
import math
from scipy import linalg, matrix
from PIL import Image

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [3]:
def get_line_2(pt1, pt2):
    points = [(pt1[0], pt1[1]),(pt2[0], pt2[1])]
    x_coords, y_coords = zip(*points)
    A = np.vstack([x_coords ,np.ones(len(x_coords))]).T
    m, c = np.linalg.lstsq(A, y_coords, rcond=-1)[0]
    return m, c

In [4]:
def get_line(pt1, pt2, pt3):
    points = [(pt1[0], pt1[1]),(pt2[0], pt2[1]), (pt3[0], pt3[1])]
    x_coords, y_coords = zip(*points)
    A = np.vstack([x_coords ,np.ones(len(x_coords))]).T
    m, c = np.linalg.lstsq(A, y_coords, rcond=-1)[0]
    return m, c

In [5]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    pixel_lines = []
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        m, c = get_line_2(pt1, pt2)
        pixel_lines.append([-m, 1, c])
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')
    l1 = pixel_lines[0]
    l2 = pixel_lines[1]
    vp = intersection(l1, l2)
    plt.plot(vp[0] , vp[1] , 'ro')
    plt.show()
    return np.array([vp[0], vp[1], 1])

In [6]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [7]:
def intersection(L1, L2): # https://stackoverflow.com/questions/20677795/how-do-i-compute-the-intersection-point-of-two-lines-in-python
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return np.array([x, y, 1])
    else:
        return [False]

In [8]:
def get_vanishing_point(n, lines, centers):
    """
    Solves for the vanishing point using the user-input lines.
    given ax+by+c, change to y = a/bx + c/b
    """
    line1 = np.reshape(lines[0], (3, ))
    line2 = np.reshape(lines[1], (3, ))
    line3 = np.reshape(lines[2], (3, ))
    line1[2] = np.negative(line1[2])
    line2[2] = np.negative(line2[2])
    line3[2] = np.negative(line3[2])
    i12 = intersection(line1, line2)
    i13 = intersection(line1, line3)
    i23 = intersection(line3, line3)
    if i12.any != False:
        return i12
    elif i13.any != False:
        return i13
    else:
        return i23

In [9]:
def get_horizon_line(vpts, pixel_vpts):
    """
    Calculates the ground horizon line.
    """
    pt1 = vpts[0]
    pt2 = vpts[1]
    pt3 = vpts[2]
    pixel_pt1 = pixel_vpts[0]
    pixel_pt2 = pixel_vpts[1]
    pixel_pt3 = pixel_vpts[2]
    p_line = get_line(pixel_pt1, pixel_pt2, pixel_pt3)
    line = get_line(pt1, pt2, pt3)
    return line, p_line

In [10]:
def plot_horizon_line(im, pt1, pt2):
    plt.figure()
    plt.imshow(im)
    plt.plot([pt1[0], pt1[1]], [pt2[0], pt2[1]], 'r')
    plt.show()
    return

In [11]:
def null(A, eps=1e-15):
    u, s, vh = scipy.linalg.svd(A)
    null_mask = (s <= eps)
    null_space = scipy.compress(null_mask, vh, axis=0)
    return scipy.transpose(null_space)

In [12]:
def get_camera_parameters(vpts):
    v1 = np.array([vpts[0][0], vpts[1][0], vpts[2][0]])
    v2 = np.array([vpts[0][1], vpts[1][1], vpts[2][1]])
    v3 = np.array([vpts[0][2], vpts[1][2], vpts[2][2]])
    a = v1[0]
    b = v1[1]
    c = v1[2]
    d = v2[0]
    e = v2[1]
    g = v2[2]
    l = v3[0]
    m = v3[1]
    n = v3[2]
    A = np.array([[math.pow(g, 2), -2*d*g, -2*e*g],[c*g,-(c*d)-(g*a),-(c*e)-(b*g)],[n*c, -(l*c)-(n*a), -(m*c)-(n*b)]])
    b = np.array([-(math.pow(e, 2))-(math.pow(d, 2)), -(d*a)-(e*b), -(c*a)-(b*m)])
    s = np.linalg.lstsq(A, b, rcond=-1)[0]
    u = s[1]
    v = s[2]
    f = math.sqrt(np.absolute(s[0] - (math.pow(u, 2)) - (math.pow(v, 2))))
    return f, u, v

In [13]:
def get_rotation_matrix(vx, vy, vz, K):
    vx = np.reshape(vx, (3, 1))
    vy = np.reshape(vy, (3, 1))
    vz = np.reshape(vz, (3, 1))
    v = np.hstack((vx, vy))
    v = np.hstack((v, vz))
    a = np.dot(np.linalg.inv(K), v)
    l = 1/(np.linalg.det(a))
    l = math.pow(l, -3)
    R = l*np.dot(np.linalg.inv(K), v)
    return R

In [14]:
def estimate_height(coords, t_0, b_0, H, vx, vy, vz):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    r = np.array([(coords)[0][0], (coords)[1][0], (coords)[2][0]])
    b = np.array([(coords)[0][1], (coords)[1][1], (coords)[2][1]])
    m1, c1 = get_line_2(b, b_0)
    m2, c2 = get_line_2(vx, vy)
    v = intersection([-m1, 1.0, c1], [-m2, 1.0, c2])
    m3, c3 = get_line_2(v, t_0)
    m4, c4 = get_line_2(b, r)
    t = intersection([-m3, 1.0, c3], [-m4, 1.0, c4])
    t[0] = b[0]
    
    i = (np.linalg.norm(np.subtract(t, b))*np.linalg.norm(np.subtract(vz, r))) / (np.linalg.norm(np.subtract(r, b))*np.linalg.norm(np.subtract(vz, t)))
    return H/i

# Main function

In [15]:
im = np.asarray(Image.open('CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
pixel_vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(n, lines, centers)
    # Plot the lines and the vanishing point
    pixel_vpts[:, i] = plot_lines_and_vp(im, lines, vpts[:, i])
    

horizon_line, pixel_line = get_horizon_line(vpts, pixel_vpts)
plot_horizon_line(im, pixel_vpts[0], pixel_vpts[1])

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)
v1 = np.array([pixel_vpts[0][0], pixel_vpts[1][0], pixel_vpts[2][0]])
v2 = np.array([pixel_vpts[0][1], pixel_vpts[1][1], pixel_vpts[2][1]])
v3 = np.array([pixel_vpts[0][2], pixel_vpts[1][2], pixel_vpts[2][2]])
print ("Pixel vanishing points: ", v1, " ", v2, " ", v3, "\n")
print ("Horizon line (non-pixel and pixel): ", horizon_line, pixel_line, "\n")
print ("Horizon line non-pixel normalized: ", "\n")
print ("F: ", f, "U: ", u, "V: ", v, "\n")
vh1 = np.array([vpts[0][0], vpts[1][0], vpts[2][0]])
vh2 = np.array([vpts[0][1], vpts[1][1], vpts[2][1]])
vh3 = np.array([vpts[0][2], vpts[1][2], vpts[2][2]])
# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(vh2, vh3, vh1, np.array([[f,0, -u/f], [0, f, -v/f], [0, 0, 1]]))
print("Rotation Matrix: ", R, "\n")
# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)
    
t = np.array([(coords['person'])[0][0], (coords['person'])[1][0], (coords['person'])[2][0]])
b = np.array([(coords['person'])[0][1], (coords['person'])[1][1], (coords['person'])[2][1]])
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(coords[obj], t, b , 72.0, v2, v3, v1)
    print(height/12, '\n')


Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 2
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the mi

/Users/jacobhutter/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in divide
/Users/jacobhutter/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in divide
/Users/jacobhutter/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in divide
/Users/jacobhutter/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in divide


('Pixel vanishing points: ', array([-157.35614692,  245.46977405,    1.        ]), ' ', array([1.56281829e+03, 1.90987698e+02, 1.00000000e+00]), ' ', array([470.5887283 , 127.19401028,   1.        ]), '\n')
('Horizon line (non-pixel and pixel): ', (1.7451247295399075, 0.3123353685624286), (-3.0491828378433765, 675.5099110826133), '\n')
('Horizon line non-pixel normalized: ', '\n')
('F: ', 16.68436663063714, 'U: ', 9.276835297685642, 'V: ', 13.148676980682714, '\n')
('Rotation Matrix: ', array([[ 6.96951455e-09,  1.40983503e-09,  2.36199646e-09],
       [-2.98747740e-09,  4.83717391e-10, -9.56273740e-10],
       [ 3.74788261e-08,  3.74788261e-08,  3.74788261e-08]]), '\n')
Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click